<a href="https://colab.research.google.com/github/Sim98B/MushroomVision/blob/main/MushD4/MushDetector4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mushroom Detector

## Setup

In [1]:

# Importing main libraries
try:
  import torch
  print(f"torch: {torch.__version__}")
  import torchvision
  print(f"torchvision: {torchvision.__version__}")
  import torchinfo
  print(f"torchinfo: {torchinfo.__version__}")
except:
  print("[INFO] Torch and Torchvision not installed, downloading")
  !pip install -q torch torchvision torchinfo
  import torch
  import torchvision
  import torchinfo

# Utilities
import os
from pathlib import Path
import requests
import shutil
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")

# Manipulation
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Neural networks
from torch import nn
from torchvision import transforms
import torchvision.models
from torchinfo import summary

# Metrics
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Cloning GitHub repository
!git clone https://github.com/Sim98B/MushroomVision
!mv MushroomVision/Modules/Libraries .

# Custom functions
from Libraries import data, extractors, model_ops, utils

shutil.rmtree("MushroomVision")

torch: 2.1.0+cu118
torchvision: 0.16.0+cu118
[INFO] Torch and Torchvision not installed, downloading
Cloning into 'MushroomVision'...
remote: Enumerating objects: 3827, done.
remote: Counting objects: 100% (572/572), done.
remote: Compressing objects: 100% (327/327), done.
remote: Total 3827 (delta 272), reused 470 (delta 214), pack-reused 3255
Receiving objects: 100% (3827/3827), 580.57 MiB | 12.06 MiB/s, done.
Resolving deltas: 100% (961/961), done.
Updating files: 100% (1739/1739), done.


In [2]:
# Device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## Data: CIFAR100

In [3]:
tensor_transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform = tensor_transform)
test_data = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform = tensor_transform)

print(f"{len(train_data)} images for train divided into {len(train_data.classes)} classes")
print(f"{len(test_data)} images for train divided into {len(test_data.classes)} classes")

100%|██████████| 169001437/169001437 [00:05<00:00, 29942154.24it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified
50000 images for train divided into 100 classes
10000 images for train divided into 100 classes


In [4]:
for i in range(len(train_data.classes)):
  if train_data.classes[i] == "mushroom":
    print(f"{i+1}. {train_data.classes[i].upper()}")
  else:
    print(f"{i+1}. {train_data.classes[i]}")

1. apple
2. aquarium_fish
3. baby
4. bear
5. beaver
6. bed
7. bee
8. beetle
9. bicycle
10. bottle
11. bowl
12. boy
13. bridge
14. bus
15. butterfly
16. camel
17. can
18. castle
19. caterpillar
20. cattle
21. chair
22. chimpanzee
23. clock
24. cloud
25. cockroach
26. couch
27. crab
28. crocodile
29. cup
30. dinosaur
31. dolphin
32. elephant
33. flatfish
34. forest
35. fox
36. girl
37. hamster
38. house
39. kangaroo
40. keyboard
41. lamp
42. lawn_mower
43. leopard
44. lion
45. lizard
46. lobster
47. man
48. maple_tree
49. motorcycle
50. mountain
51. mouse
52. MUSHROOM
53. oak_tree
54. orange
55. orchid
56. otter
57. palm_tree
58. pear
59. pickup_truck
60. pine_tree
61. plain
62. plate
63. poppy
64. porcupine
65. possum
66. rabbit
67. raccoon
68. ray
69. road
70. rocket
71. rose
72. sea
73. seal
74. shark
75. shrew
76. skunk
77. skyscraper
78. snail
79. snake
80. spider
81. squirrel
82. streetcar
83. sunflower
84. sweet_pepper
85. table
86. tank
87. telephone
88. television
89. tiger
90. 

### Creating a binary dataset

In [5]:
train_data = utils.binary_dataset(dataset = train_data,
                                  target_classes = 51)

test_data = utils.binary_dataset(dataset = test_data,
                                 target_classes = 51)

print(f"Train non-mushrooms images: {train_data.targets.count(0)}, mushrooms images: {train_data.targets.count(1)}")
print(f"Test non-mushrooms images: {test_data.targets.count(0)}, mushrooms images: {test_data.targets.count(1)}")

class_names = ["Non mushroom", "Mushroom"]

Old classes: 100
New classes: 2
Old classes: 100
New classes: 2
Train non-mushrooms images: 49500, mushrooms images: 500
Test non-mushrooms images: 9900, mushrooms images: 100


## Setting up experiments

In [6]:
ALEXNET, alexnet_transformer = extractors.create_alexnet(output_shape = len(class_names))
VGG, vgg_transformer = extractors.create_vgg16(output_shape = len(class_names))
DENSENET, densenet_transformer = extractors.create_densenet121(output_shape = len(class_names))
RESNET, resnet_transformer = extractors.create_resnet50(output_shape = len(class_names))

#for model in [ALEXNET, VGG, DENSENET, RESNET]:
#  if model.__class__.__name__ != "ResNet":
#    print(f"{model.__class__.__name__} params:\nTOTAL: {summary(model, input_size = (32, 3, 256, 256)).total_params:,}\nTRAINABLE: {summary(model, input_size = (32, 3, 256, 256)).trainable_params:,}")
#  else:
#    print(f"{model.__class__.__name__} params:\nTOTAL: {summary(model, input_size = (32, 3, 232, 232)).total_params:,}\nTRAINABLE: {summary(model, input_size = (32, 3, 232, 232)).trainable_params:,}")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 79.2MB/s]
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 80.6MB/s]
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 79.0MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 158MB/s]


In [7]:
utils.set_seed()

experiment_number = 0

for epoch in [5, 10, 15, 20]:

  for extractor in [ALEXNET, VGG, DENSENET, RESNET]:
    experiment_number += 1
    print(f"Experiment n°{experiment_number}, model: {extractor.__class__.__name__}, {epoch} epochs")

    if extractor == ALEXNET:

      model, transformer = extractors.create_alexnet(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "f1",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 1)

    elif extractor == VGG:

      model, transformer = extractors.create_vgg16(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "f1",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 1)

    elif extractor == DENSENET:

      model, transformer = extractors.create_densenet121(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "f1",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 1)

    else:

      model, transformer = extractors.create_resnet50(output_shape = len(class_names))
      train_data.transform = transformer
      test_data.transform = transformer
      experiment_train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                                            batch_size = 32,
                                                            shuffle = True,
                                                            num_workers = os.cpu_count(),
                                                            pin_memory = True)
      experiment_val_loader = torch.utils.data.DataLoader(dataset = test_data,
                                                          batch_size = 32,
                                                          shuffle = False,
                                                          num_workers = os.cpu_count(),
                                                          pin_memory = True)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(params = model.parameters(), lr=0.001)

      model_ops.train(model = model,
                      train_dataloader = experiment_train_loader,
                      test_dataloader = experiment_val_loader,
                      loss_function = criterion,
                      optimizer = optimizer,
                      metric = "f1",
                      epochs = epoch,
                      device = device,
                      writer = utils.create_writer(experiment_name = f"{epoch}_epochs",
                                                   model_name = extractor.__class__.__name__),
                      verbose = 1)

Experiment n°1, model: AlexNet, 5 epochs
[INFO] Created SummaryWriter, saving to: runs/2023-11-24/5_epochs/AlexNet...


Training:   0%|          | 0/5 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
a = torch.Tensor([1,2,3]).to("cuda")
b = torch.Tensor([1,2,2]).to("cuda")

f1_score(a.cpu(),b.cpu(), average = "weighted")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs